# World's Largest Urban Areas Explorer
## Source: https://en.wikipedia.org/wiki/List_of_urban_areas_by_population

See wikimining.py to see how wikiurbanareas.csv was created.
This is only visible with plotly installed and within jupyter notebook itself.

In [1]:
from plotly.widgets import GraphWidget
from plotly.graph_objs import *
from ipywidgets import *
from IPython.display import display, clear_output, display_html
import pandas as pd
import numpy as np

<IPython.core.display.Javascript object>

In [2]:
#import wikidata into pandas
bigcities = pd.read_csv('wikiurbancities.csv', encoding = 'ISO-8859-1')
statlist = [(0,'Population','Population','blue'),
            (1,'Area_km2','Area (km^2)','red'),
            (2,'Density_km2','Density (/km^2)','purple')]

In [3]:
#load widgets
earth = GraphWidget()
bars = GraphWidget()
elayout = Layout(title = "World's largest urban areas",geo=dict(projection=dict(type='orthographic')))
blayout = Layout()

stat = Dropdown()
stat.description = 'Select a statistic: '
stat.options = ['']
stat.value = ''

ncities = IntSlider()
ncities.description = 'Cities to display: '
ncities.min = 1
ncities.max = 497
ncities.value = 200

mapb = Button()
mapb.description = 'Show World'


def mapIt(b):
    #Blank charts must load first for earth to add plots
    if mapb.description == 'Show World':
        stat.options = {x[2]:x[0] for x in statlist}
        stat.value = 0
        mapb.description = 'Map it'
        earth.plot({})
        edata = [Scattergeo()]
        efig = Figure(data=edata, layout=elayout)
        earth.plot(efig)
        bdata = [Bar()]
        bfig = Figure(data=bdata, layout=blayout)
        bars.plot(bfig)
        
    #All other transformations occur here
    else:
        graphcities = bigcities.loc[:,[statlist[stat.value][1],'Search','Latitude','Longitude']].sort(
            statlist[stat.value][1], ascending = False)[:ncities.value]
        size = ((2*(graphcities[statlist[stat.value][1]] - np.mean(bigcities[statlist[stat.value][1]]))
        /np.std(bigcities[statlist[stat.value][1]])) + 10)
        text = []
        for i in range(len(graphcities)):
            thistext = (graphcities.iloc[i,1] + ', ' + statlist[stat.value][2] 
                        + ': ' + format(int(graphcities.iloc[i,0]), ",d"))
            text.append(thistext)
        edata = [Scattergeo(lat = graphcities.Latitude,
                                    lon = graphcities.Longitude,
                                    marker = Marker(size=size,
                                                   color=statlist[stat.value][3]),
                                    text = text)]
        efig = Figure(data=edata, layout=elayout)
        earth.plot(efig)
        bars.delete_traces(0)
        bars.add_traces([Bar(x = graphcities.Search, y = graphcities.iloc[:,0],
                             marker = dict(color = statlist[stat.value][3]))])
        bars.relayout({'title': 'Top ' + str(ncities.value) + ' Urban Areas by ' + statlist[stat.value][2]})
        bdata = [Bar(x = graphcities.Search, y = graphcities.iloc[:,0], marker = dict(color = statlist[stat.value][3]))]
        bfig = Figure(data=bdata, layout=blayout)
        #bars.plot(bfig)

mapb.on_click(mapIt)

display(stat)
display(ncities)
display(mapb)
display(earth)
display(bars)